In [3]:
import os
import numpy as np

import IPython.display as ipd

from pedalboard import Compressor, Gain, Pedalboard, Reverb, Distortion, Delay, time_stretch, PitchShift # , Limiter, NoiseGate,  

import librosa
import numpy as np

import pyloudnorm as pyln

import soundfile as sf
import json
# import to save as mp3
import pydub


import pandas as pd

In [19]:
def load_audio_files(dir, ext=".mp3"):
    audio_files = []
    for file in os.listdir(dir):
        if file.endswith(ext):
            audio_files.append(os.path.join(dir, file))
    return audio_files

sr = 44100

base_track_length_s =60

# load base tracks from /home/ivan/Documents/FIng/QueenMary/
base_tracks_paths = load_audio_files("/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/base tracks/")
base_tracks = []
for track in base_tracks_paths:
    print(f"Loading {track}...")
    y, _ = librosa.load(track, sr=sr)
    # # cut base song to base_track_length_s
    y = y[:base_track_length_s*sr]
    base_tracks.append(y)

Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/base tracks/HipHop.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/base tracks/EDM.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/base tracks/Pop.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/base tracks/edmStable.mp3...


In [20]:
samples_path = "/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/"
sample_tracks_paths = load_audio_files(samples_path, ext = ".wav")
sample_tracks = []
for track in sample_tracks_paths:
    print(f"Loading {track}...")
    y, _ = librosa.load(track, sr=sr)
    # pad song with 1s of silence before and after
    print(y.shape)
    y = np.concatenate([np.zeros(4*sr), y, np.zeros(int(0.2*sr))])
    print(y.shape)
    sample_tracks.append(y)


Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S128_T029.wav...
(450432,)
(635652,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S131_T003.wav...
(474880,)
(660100,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S069_T112.wav...
(675840,)
(861060,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S050_T085.wav...
(255360,)
(440580,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S037_T060.wav...
(487424,)
(672644,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S008_T014.wav...
(464256,)
(649476,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S005_T007.wav...
(434176,)
(619396,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S112_T088.wav...
(405504,)
(590724,)
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S0

In [21]:
cfg = {
    "sample_start": {
        "seconds": None, # 0,
        "beats": 0#24
    },
    "sample_length": {
        "seconds": None, # 5,
        "beats": 16#16
    },
    "gain": {
        "gain_db": 0 #-3
    },
    "effects":  {
        "compressor": False, #{
        #     "threshold_db": -6,
        #     "ratio": 6,
        #     "attack_ms": 0.001,
        #     "release_ms": 0.2
        # },
        "reverb": False, #{
        #     "room_size": 0.8,
        #     "damping": 0.1,
        #     "wet_level": 0.5,
        #     "dry_level": 0.5
        # },
        "delay": False, #{
            #     "delay_seconds": 0.3,
            #     "feedback": 0.4,
            #     "mix": 0.3
            # },
        "noise_gate": False, #{
        #     "threshold_db": -10,
        #     "attack_ms": 0.001,
        #     "release_ms": 0.1
        # },
        "distortion": False, #{
        #     "drive_db": 20
        # },
        "limiter": False, #{
        #     "threshold_db": -3
        # },
        "noise_gate": False, #{
        #     "threshold_db": -40,
        #     "attack_ms": 0.001,
        #     "release_ms": 0.1
        # },
    },
    "beat_sync": {
        "start_beat": 8, # 0 is the first beat
    },
    "tempo_sync": {
        "multiplier": 0.95 # 1 is the original tempo of the base track
    },
    "tempo_change": 1, # 1 is no change
    "pitch_sync": True, # detects key of the base track and adjusts pitch of sample to match
    "pitch_shift": 0, # in semitones
    "looping": {
        "n_loops": 2, # 1 is no loop
        "loop_every_n_beats": 12
    },
    "masking": False,
}
def effect_sample(sample, sr, cfg):
    assert cfg["gain"] is not None, "Gain is required"
    gain = Gain(gain_db=cfg["gain"]["gain_db"])
    pedalboard = Pedalboard([gain])
    if cfg["effects"]:
        if cfg["effects"]["compressor"]:
            compressor = Compressor(threshold_db=cfg["effects"]["compressor"]["threshold_db"], ratio=cfg["effects"]["compressor"]["ratio"], attack_ms=cfg["effects"]["compressor"]["attack_ms"], release_ms=cfg["effects"]["compressor"]["release_ms"])
            pedalboard.append(compressor)
        if cfg["effects"]["reverb"]:
            reverb = Reverb(room_size=cfg["effects"]["reverb"]["room_size"], damping=cfg["effects"]["reverb"]["damping"], wet_level=cfg["effects"]["reverb"]["wet_level"], dry_level=cfg["effects"]["reverb"]["dry_level"])
            pedalboard.append(reverb)
        # if cfg["effects"]["limiter"]:
        #     limiter = Limiter(threshold_db=cfg["effects"]["limiter"]["threshold_db"])
        #     pedalboard.append(limiter)
        # if cfg["effects"]["noise_gate"]:
        #     noise_gate = NoiseGate(threshold_db=cfg["effects"]["noise_gate"]["threshold_db"], attack_ms=cfg["effects"]["noise_gate"]["attack_ms"], release_ms=cfg["effects"]["noise_gate"]["release_ms"])
        #     pedalboard.append(noise_gate)
        if cfg["effects"]["delay"]:
            delay = Delay(delay_seconds=cfg["effects"]["delay"]["delay_seconds"], feedback=cfg["effects"]["delay"]["feedback"], mix=cfg["effects"]["delay"]["mix"])
            pedalboard.append(delay)
        if cfg["effects"]["distortion"]:
            distortion = Distortion(drive_db=cfg["effects"]["distortion"]["drive_db"])
            pedalboard.append(distortion)

    return pedalboard.process(sample, sr)


def loudness_normalize(y, sr, target_loudness=-12.0):
    # peak_normalized_y = pyln.normalize.peak(y, -1.0)
    meter = pyln.Meter(sr, block_size=0.200) # create BS.1770 meter, default block size is 400ms
    loudness = meter.integrated_loudness(y)
    y = pyln.normalize.loudness(y, loudness, target_loudness)
    return y

In [22]:
# create song by adding sample to base
def create_song(base, sample_source, cfg=None):
    # print(f"Peak value of base: {np.max(base)}, Peak value of sample: {np.max(sample)}")
    sample_tempo, sample_beats = librosa.beat.beat_track(y=sample_source, sr=sr)
    print(f"Sample tempo: {sample_tempo}")
    base_tempo, base_beats = librosa.beat.beat_track(y=base, sr=sr)
    assert cfg is not None, "Configuration is required"

    sample_time_annotation = []
    base_time_annotations = []

    # extract sample from source
    print("Extracting sample from sample source...")
    assert cfg["sample_start"] is not None, "Sample start is required"
    assert cfg["sample_length"] is not None, "Sample length is required"
    if cfg["sample_start"]["seconds"] is not None:
        sample_source = sample_source[cfg["sample_start"]["seconds"]*sr:]
        sample_time_annotation.append({"start_time": cfg["sample_start"]["seconds"], "end_time": None})
    elif cfg["sample_start"]["beats"] is not None:
        sample_source = sample_source[librosa.frames_to_samples(sample_beats[cfg["sample_start"]["beats"]]):]
        sample_time_annotation.append({"start_time": librosa.frames_to_time(sample_beats[cfg["sample_start"]["beats"]], sr=sr), "end_time": None})
        sample_beats -= sample_beats[cfg["sample_start"]["beats"]]
    if cfg["sample_length"]["seconds"] is not None:
        sample = sample_source[:cfg["sample_length"]["seconds"]*sr]
        sample_time_annotation[-1]["end_time"] = cfg["sample_length"]["seconds"]
    elif cfg["sample_length"]["beats"] is not None:
        try:
            sample = sample_source[:librosa.frames_to_samples(sample_beats[cfg["sample_start"]["beats"]+cfg["sample_length"]["beats"]])]
            sample_time_annotation[-1]["end_time"] = sample_time_annotation[-1]["start_time"] + librosa.frames_to_time(sample_beats[cfg["sample_start"]["beats"]+cfg["sample_length"]["beats"]], sr=sr)
        except IndexError:
            print("Sample length exceeds sample source length. Truncating sample to end of source.")
            sample = sample_source[:]
            sample_time_annotation[-1]["end_time"] = sample_time_annotation[-1]["start_time"] + librosa.frames_to_time(len(sample), sr=sr)
            
    # loudness normalize audio to -12 dB LUFS
    base = loudness_normalize(base, sr, target_loudness=-12.0)
    sample = loudness_normalize(sample, sr, target_loudness=-12.0)
    original_sample = sample

    # half the volume
    gain = Gain(-6)

    board = Pedalboard([gain])

    base = board.process(base, sr)
    sample = board.process(sample, sr)

    # base_start_sample = 0
    pitch_shift_semitones = 0

    # if cfg["masking"]:
    #     # take the signal to silence for n beats
    #     pass

    if cfg["tempo_sync"]:
        stretch_rate = (base_tempo/sample_tempo)[0] * cfg["tempo_sync"]["multiplier"]
        cfg["total_tempo_stretch_rate"] = float(stretch_rate)
        sample = time_stretch(sample, sr, stretch_rate)[0]

    if cfg["tempo_change"] != 1:
        sample = time_stretch(sample, sr, cfg["tempo_change"])[0]

    if cfg["pitch_sync"]:
        sample_chromagram = librosa.feature.chroma_stft(y=sample, sr=sr)
        base_chromagram = librosa.feature.chroma_stft(y=base, sr=sr)
        mean_sample_chroma = np.mean(sample_chromagram, axis=1)
        mean_base_chroma = np.mean(base_chromagram, axis=1)
        sample_estimated_key = np.argmax(mean_sample_chroma)
        base_estimated_key = np.argmax(mean_base_chroma)
        pitch_shift_semitones = base_estimated_key - sample_estimated_key

    if cfg["pitch_shift"]:
        pitch_shift_semitones += cfg["pitch_shift"]

    cfg["total_pitch_shift"] = int(pitch_shift_semitones)

    pitch_shift = Pedalboard([PitchShift(semitones=pitch_shift_semitones)])
    sample = pitch_shift.process(sample, sr)

        
    print("Applying effects to sample...")
    sample = effect_sample(sample, sr, cfg)
    effected_sample = sample

    mix = base
    # mix[:len(sample)] += sample
    assert cfg["looping"] is not None, "Looping configuration is required (even if n_loops=1)"
    start_beat = cfg["beat_sync"]["start_beat"]
    for _ in range(cfg["looping"]["n_loops"]):
        loop_start_sample = librosa.frames_to_samples(base_beats[start_beat])
        mix[loop_start_sample:loop_start_sample+len(sample)] += sample

        start_beat += cfg["looping"]["loop_every_n_beats"]

        base_time_annotations.append({"start_time": librosa.frames_to_time(base_beats[start_beat], sr=sr), "end_time": librosa.frames_to_time(base_beats[start_beat]+librosa.samples_to_frames(len(sample)), sr=sr)})

        if cfg["looping"]["n_loops"] != 1 and cfg["looping"]["loop_every_n_beats"] < cfg["sample_length"]["beats"]:
            print ("Warning: Loops will overlap as sample_length in beats is greater than loop_every_n_beats")

        # # # find closest beat to start_sample
        # start_beat += cfg["looping"]["loop_every_n_beats"]
        # # loop_start_sample = librosa.frames_to_samples(base_beats[end_beat + cfg["looping"]["loop_every_n_beats"]])
        # loop_start_sample = librosa.frames_to_samples(base_beats[start_beat])


    mix[len(sample):] += base[len(sample):]
    # mix[base_start_sample+len(sample):] += base[base_start_sample+len(sample):]

    mix = loudness_normalize(mix, sr, target_loudness=-12.0)

    return mix, original_sample, effected_sample, base_time_annotations, sample_time_annotation, cfg

In [23]:
sample_tracks

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [ ]:
save_files = False
version = 6

# sample_tracks = sample_tracks[1:]

for i, (base, sample) in enumerate(zip(base_tracks[:4], sample_tracks[:4])):
    mix, original_sample, effected_sample, base_time_annotations, sample_time_annotation, cfg = create_song(base, sample, cfg)
    print("====================================")
    print("Original sample")
    ipd.display(ipd.Audio(original_sample, rate=sr))
    print("Sample after processing")
    ipd.display(ipd.Audio(effected_sample, rate=sr))
    print("Mix (final song)")
    ipd.display(ipd.Audio(mix, rate=sr))

    if save_files:
        # save mixed audio together with config used
        # save audio as mp3
        sf.write(f"mix_{i}_{i*2}_v{version}.wav", mix, sr)
        # convert to mp3
        sound = pydub.AudioSegment.from_wav(f"mix_{i}_{i*2}_v{version}.wav")
        sound.export(f"mix_{i}_{i*2}_v{version}.mp3", format="mp3")

    # add time annotations to config
    cfg["base_time_annotations"] = base_time_annotations
    cfg["sample_time_annotations"] = sample_time_annotation
    # add file name to config
    cfg["base_file"] = base_tracks_paths[i].split("/")[-1]
    cfg["sample_file"] = sample_tracks_paths[i*2].split("/")[-1]
    if save_files:
        # save config
        with open(f"config_{i}_{i*2}_v{version}.json", "w") as f:
            json.dump(cfg, f)
    else:
        print(cfg)
    print("====================================")

In [2]:
cfg = {
    "sample_start": {
        "seconds": None, # 0,
        "beats": 0#24
    },
    "sample_length": {
        "seconds": None, # 5,
        "beats": 16
    },
    "gain": {
        "gain_db": 0
    },
    "effects":  {
        "compressor": False, 
        "reverb": False, 
        "delay": False, 
        "noise_gate": False,
        "distortion": False,
        "limiter": False, 
        "noise_gate": False, 
    },
    "beat_sync": {
        "start_beat": 12, # 0 is the first beat
    },
    "tempo_sync": False, 
    "tempo_change": 1, # 1 is no change
    "pitch_sync": False, # detects key of the base track and adjusts pitch of sample to match
    "pitch_shift": 0, # in semitones
    "looping": {
        "n_loops": 2, # 1 is no loop
        "loop_every_n_beats": 16
    }
}

# [4 suno base tracks (Pop, HipHop/Rap, EDM-instrumental, House-instrumental) + 1 stableAudio base track (EDM-instrumental)] x 10 samples from sample100 sample set (4 drum-based beats, 4 instrumental riffs, 2 with vocals) x 30 augmentations
# = 1500 tracks
config_array = [{'gain': {'gain_db': 0}},
                {'gain': {'gain_db': -3}},
                {'gain': {'gain_db': -6}},
                {'gain': {'gain_db': 1}},
                {"sample_length": {
                    "seconds": None, # 5,
                    "beats": 4
                }},
                {"sample_length": {
                    "seconds": None, # 5,
                    "beats": 8
                }},
                {'tempo_sync': {'multiplier': 1}},
                {'tempo_change': 0.98},
                {'tempo_change': 1.02},
                {'tempo_change': 1.05},
                {'tempo_change': 1.5},
                {'tempo_change': 2},
                {'pitch_sync': True},
                {'pitch_shift': -1},
                {'pitch_shift': 1},
                {'pitch_shift': -3},
                {'pitch_shift': 3},
                {'pitch_shift': -7},
                {'pitch_shift': 7},
                {'pitch_shift': -12},
                {'pitch_shift': 12},
                {'pitch_sync': True, 'tempo_sync': {'multiplier': 1}},
                {'effects': {'compressor': {'threshold_db': -6, 'ratio': 6, 'attack_ms': 0.001, 'release_ms': 0.2}, 'reverb': False, 'delay': False, 'distortion': False}},
                {'effects': {'reverb': {'room_size': 0.8, 'damping': 0.1, 'wet_level': 0.5, 'dry_level': 0.5}, 'compressor': False, 'delay': False, 'distortion': False}},
                {'effects': {'delay': {'delay_seconds': 0.3, 'feedback': 0.4, 'mix': 0.3}, 'compressor': False, 'reverb': False, 'distortion': False}},
                {'effects': {'distortion': {'drive_db': 20},'compressor': False, 'reverb': False, 'delay': False}}, 
                {'effects': {'compressor': {'threshold_db': -6, 'ratio': 6, 'attack_ms': 0.001, 'release_ms': 0.2},
                            'reverb': {'room_size': 0.8, 'damping': 0.1, 'wet_level': 0.5, 'dry_level': 0.5},
                            'delay': {'delay_seconds': 0.3, 'feedback': 0.4, 'mix': 0.3},
                            'distortion': {'drive_db': 20}}
                            },
                # {'drums_separation': True},
                # {'vocal_separation': True},
                # {'bass_separation': True}
                ]

print(f"Creating {len(config_array)} songs...")
                
                

Creating 27 songs...


In [25]:
# load mp3 files that contain stem separated samples
separated_samples_path = "/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/"
separated_sample_tracks_paths = load_audio_files(separated_samples_path, ext = ".mp3")
separated_sample_tracks = []
for track in separated_sample_tracks_paths:
    print(f"Loading {track}...")
    y, _ = librosa.load(track, sr=sr)
    # pad song with 1s of silence before and after
    y = np.concatenate([np.zeros(4*sr), y, np.zeros(int(0.2*sr))])
    separated_sample_tracks.append(y)

Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S008_T014_mvsep_drums_model_mt_3_drums.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S005_T007_mvsep_bass_model_mt_1_bass.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S003_T003_mvsep_bass_model_mt_1_bass.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S005_T007_mvsep_drums_model_mt_3_drums.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S128_T029_bs_roformer_mt_29_vocals.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S069_T112_bs_roformer_mt_29_vocals.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S003_T003_mvsep_drums_model_mt_3_drums.mp3...
Loading /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S128_T029_mvsep_bass_model_mt_1_bass.mp3...
Loading /home/ivanmere

In [ ]:
# create folder to save songs
import soundfile as sf
import json
# import to save as mp3
import pydub
import os
# go to file directory
os.chdir("/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/")
os.makedirs(f"artificial_dataset_v2", exist_ok=True)
os.chdir(f"artificial_dataset_v2")
os.makedirs("audio", exist_ok=True)
os.makedirs("audio_wav", exist_ok=True)

# all the base-samples combinations
version = 0
for config in config_array:
    for base_index, base_track in enumerate(base_tracks):
        for sample_index, sample_track in enumerate(sample_tracks):
            aux_cfg = cfg.copy()
            aux_cfg.update(config)
            mix, original_sample, effected_sample, base_time_annotations, sample_time_annotation, aux_cfg = create_song(base_track, sample_track, aux_cfg)

            # save audio together with config used
            # save audio as wav
            sf.write(f"audio_wav/mix_{base_index}_{sample_index}_v{version}.wav", mix, sr)
            sf.write(f"audio_wav/original_sample_{base_index}_{sample_index}_v{version}.wav", original_sample, sr)
            sf.write(f"audio_wav/effected_sample_{base_index}_{sample_index}_v{version}.wav", effected_sample, sr)
            # convert to mp3
            sound = pydub.AudioSegment.from_wav(f"audio_wav/mix_{base_index}_{sample_index}_v{version}.wav")
            sound.export(f"audio/mix_{base_index}_{sample_index}_v{version}.mp3", format="mp3")
            print(f"Created mix_{base_index}_{sample_index}_v{version}.mp3")
            sound = pydub.AudioSegment.from_wav(f"audio_wav/original_sample_{base_index}_{sample_index}_v{version}.wav")
            sound.export(f"audio/original_sample_{base_index}_{sample_index}_v{version}.mp3", format="mp3")
            print(f"Created original_sample_{base_index}_{sample_index}_v{version}.mp3")
            sound = pydub.AudioSegment.from_wav(f"audio_wav/effected_sample_{base_index}_{sample_index}_v{version}.wav")
            sound.export(f"audio/effected_sample_{base_index}_{sample_index}_v{version}.mp3", format="mp3")
            print(f"Created effected_sample_{base_index}_{sample_index}_v{version}.mp3")

            # add time annotations to config
            aux_cfg["base_time_annotations"] = base_time_annotations
            aux_cfg["sample_time_annotations"] = sample_time_annotation
            # add base file name to config
            aux_cfg["base_file"] = base_tracks_paths[base_index]
            aux_cfg["sample_file"] = sample_tracks_paths[sample_index]
            # save config
            with open(f"config_{base_index}_{sample_index}_v{version}.json", "w") as f:
                json.dump(aux_cfg, f)
                print("Config saved")

            print(f"Song {base_index}_{sample_index}_v{version} created!")

    version += 1

# all the base-separated_samples combinations
for base_index, base_track in enumerate(base_tracks):
    for sample_index, sample_track in enumerate(separated_sample_tracks):
        aux_cfg = cfg.copy()
        mix, original_sample, effected_sample, base_time_annotations, sample_time_annotation, aux_cfg = create_song(base_track, sample_track, aux_cfg)

        # save audio together with config used
        # save audio as wav
        sf.write(f"audio_wav/mix_{base_index}_{sample_index}_v{version}.wav", mix, sr)
        sf.write(f"audio_wav/original_sample_{base_index}_{sample_index}_v{version}.wav", original_sample, sr)
        sf.write(f"audio_wav/effected_sample_{base_index}_{sample_index}_v{version}.wav", effected_sample, sr)
        # convert to mp3
        sound = pydub.AudioSegment.from_wav(f"audio_wav/mix_{base_index}_{sample_index}_v{version}.wav")
        sound.export(f"audio/mix_{base_index}_{sample_index}_v{version}.mp3", format="mp3")
        sound = pydub.AudioSegment.from_wav(f"audio_wav/original_sample_{base_index}_{sample_index}_v{version}.wav")
        sound.export(f"audio/original_sample_{base_index}_{sample_index}_v{version}.mp3", format="mp3")
        sound = pydub.AudioSegment.from_wav(f"audio_wav/effected_sample_{base_index}_{sample_index}_v{version}.wav")
        sound.export(f"audio/effected_sample_{base_index}_{sample_index}_v{version}.mp3", format="mp3")

        # add time annotations to config
        aux_cfg["base_time_annotations"] = base_time_annotations
        aux_cfg["sample_time_annotations"] = sample_time_annotation
        # add base file name to config
        aux_cfg["base_file"] = base_tracks_paths[base_index]
        aux_cfg["sample_file"] = separated_sample_tracks_paths[sample_index]
        # save config
        with open(f"config_{base_index}_{sample_index}_v{version}.json", "w") as f:
            json.dump(aux_cfg, f)

        print(f"Song {base_index}_{sample_index}_v{version} created!")

    version += 1


NameError: name 'config_array' is not defined

In [27]:
# save df that maps version to config used
df = pd.DataFrame(config_array)
# add version column
df["version"] = list(range(len(config_array)))
# add separated_samples as extra rows of augmentation
df = pd.concat([df]*len(base_tracks)*len(separated_sample_tracks), ignore_index=True)
df.to_csv("config_versions.csv", index=False)

# save df that maps base and sample files to index
df = pd.DataFrame({"base_files": base_tracks_paths})
df["base_index"] = df.index
# concatenate with sample files
df2 = pd.DataFrame({"sample_files": sample_tracks_paths})
df2["sample_index"] = df2.index
# and separated sample files
df3 = pd.DataFrame({"sample_files": separated_sample_tracks_paths})
df3["separated_sample_index"] = df3.index
# concatenate
df = pd.concat([df, df2, df3], axis=1)

df.to_csv("base_sample_files.csv", index=False)




In [ ]:
# Ok, now I want to create a subset of the artificial dataset which I moved to /home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2/
import shutil
# create folder to save subset
os.makedirs(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset", exist_ok=True)
# create folder to save subset audio
os.makedirs(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/audio", exist_ok=True)

# move the "mix" mp3 files to the subset folder
for file in os.listdir("/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2/audio"):
    if "mix" in file:
        shutil.move(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2/audio/{file}", f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/audio/{file}")

# and for a random 25% i want to move the effect and original sample files
import random
files = os.listdir("/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2/audio")
# keep only files that are effected or original samples
files = [file for file in files if "effected" in file or "original" in file]
print(len(files))

# seed 42
random.seed(42)
random.shuffle(files)
files = files[:int(len(files)*0.25)]

for file in files:
    shutil.move(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2/audio/{file}", f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/audio/{file}")

    # and the config files for those files
    shutil.move(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2/config_{file.split('_')[2]}_{file.split('_')[3]}_{file.split('_')[4].split('.')[0]}.json", f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/config_{file.split('_')[2]}_{file.split('_')[3]}_{file.split('_')[4].split('.')[0]}.json")
    


In [4]:
# the json files contain the following information:
# "base_file": "/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/base tracks/EDM.mp3", "sample_file": "/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/samples/S003_T003.wav"
# iterate through them and keep only the file name, without the path
for file in os.listdir("/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset"):
    if "config" in file:
        with open(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/{file}", "r") as f:
            data = json.load(f)
            data["base_file"] = data["base_file"].split("/")[-1]
            data["sample_file"] = data["sample_file"].split("/")[-1]
        with open(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/{file}", "w") as f:
            json.dump(data, f)
        print(f"Fixed {file}")

Fixed config_3_3_v20.json
Fixed config_3_8_v5.json
Fixed config_1_3_v13.json
Fixed config_1_3_v25.json
Fixed config_1_0_v5.json
Fixed config_0_6_v19.json
Fixed config_0_6_v5.json
Fixed config_1_4_v5.json
Fixed config_2_1_v17.json
Fixed config_0_3_v15.json
Fixed config_3_6_v6.json
Fixed config_3_1_v14.json
Fixed config_1_6_v15.json
Fixed config_0_5_v5.json
Fixed config_1_2_v14.json
Fixed config_2_7_v24.json
Fixed config_2_4_v23.json
Fixed config_0_3_v2.json
Fixed config_0_5_v11.json
Fixed config_1_5_v2.json
Fixed config_1_6_v20.json
Fixed config_1_10_v28.json
Fixed config_0_4_v3.json
Fixed config_0_5_v27.json
Fixed config_3_6_v4.json
Fixed config_3_7_v20.json
Fixed config_0_8_v16.json
Fixed config_1_4_v18.json
Fixed config_0_8_v27.json
Fixed config_3_4_v1.json
Fixed config_2_4_v9.json
Fixed config_0_4_v20.json
Fixed config_1_1_v21.json
Fixed config_3_0_v5.json
Fixed config_2_6_v11.json
Fixed config_3_4_v2.json
Fixed config_1_2_v22.json
Fixed config_2_0_v4.json
Fixed config_3_3_v1.json
F

In [6]:
# create audio folder
os.makedirs(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/audio", exist_ok=True)
# reformat all the wav files to mp3
for file in os.listdir("/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/audio_wav"):
    sound = pydub.AudioSegment.from_wav(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/audio_wav/{file}")
    sound.export(f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/artificial_dataset_v2_subset/audio/{file.split('.')[0]}.mp3", format="mp3")
    print(f"Converted {file}")

Converted mix_3_2_v24.wav
Converted mix_3_6_v19.wav
Converted mix_3_0_v4.wav
Converted mix_2_6_v8.wav
Converted original_sample_3_5_v8.wav
Converted effected_sample_0_7_v27.wav
Converted mix_3_2_v30.wav
Converted mix_0_1_v13.wav
Converted mix_3_8_v16.wav
Converted effected_sample_1_2_v11.wav
Converted effected_sample_0_5_v22.wav
Converted mix_1_7_v0.wav
Converted original_sample_2_4_v29.wav
Converted mix_3_0_v10.wav
Converted mix_3_1_v17.wav
Converted mix_0_2_v2.wav
Converted mix_3_6_v30.wav
Converted mix_0_3_v5.wav
Converted mix_0_8_v20.wav
Converted original_sample_2_7_v5.wav
Converted original_sample_3_2_v4.wav
Converted mix_3_0_v25.wav
Converted mix_3_6_v24.wav
Converted mix_2_7_v29.wav
Converted mix_2_1_v7.wav
Converted effected_sample_2_8_v19.wav
Converted mix_3_4_v24.wav
Converted effected_sample_3_3_v1.wav
Converted effected_sample_0_2_v20.wav
Converted mix_1_10_v28.wav
Converted mix_0_9_v27.wav
Converted original_sample_0_9_v27.wav
Converted mix_3_0_v8.wav
Converted original_s

In [19]:
# open sample.csv, and for each unique sample_track, provide a SXXX label

# load the csv
df = pd.read_csv("/home/ivanmeresman-higgs/Documents/QueenMary/sample_100/samples.csv", header=0, encoding='latin-1')

# remove S029 which is contained in S124
df = df[df["sample_id"] != "S029"]
# remove S032 which is actually missing AND SHOULDNT!
df = df[df["sample_id"] != "S032"]
# remove S134 which is contained in S094
df = df[df["sample_id"] != "S134"]

# get unique sample tracks
unique_sample_tracks = df["sample_track_id"].unique()
print(len(unique_sample_tracks))

# retrieve one sample_id corresponding to each track from df
sample_ids = []
for track in unique_sample_tracks:
    sample_ids.append(df[df["sample_track_id"] == track]["sample_id"].values[0])
    print(f"Added {df[df['sample_track_id'] == track]['sample_id'].values[0]}")

print(len(sample_ids))
import random
import shutil
# randomly pick 25% of the sample_ids
random.seed(42)
random.shuffle(sample_ids)
sample_ids = sample_ids[:int(len(sample_ids)*0.25)]
print(len(sample_ids))

# copy the files to the subset folder
for sample_id in sample_ids:
    shutil.copy(f"/home/ivanmeresman-higgs/Documents/QueenMary/sample100_adi_annotations/{sample_id}.json", f"/home/ivanmeresman-higgs/Documents/QueenMary/tracks_for_stage/sample100/{sample_id}.json")
    print(f"Copied {sample_id}")



74
Added S001
Added S002
Added S004
Added S005
Added S006
Added S007
Added S008
Added S009
Added S010
Added S012
Added S013
Added S014
Added S015
Added S016
Added S017
Added S018
Added S019
Added S020
Added S021
Added S023
Added S024
Added S025
Added S027
Added S033
Added S035
Added S036
Added S037
Added S038
Added S039
Added S040
Added S041
Added S042
Added S043
Added S044
Added S045
Added S046
Added S047
Added S050
Added S051
Added S054
Added S055
Added S056
Added S060
Added S061
Added S062
Added S063
Added S064
Added S067
Added S068
Added S071
Added S073
Added S074
Added S078
Added S079
Added S080
Added S081
Added S082
Added S083
Added S084
Added S086
Added S091
Added S092
Added S094
Added S098
Added S101
Added S103
Added S104
Added S106
Added S109
Added S113
Added S114
Added S115
Added S116
Added S124
74
18
Copied S103
Copied S021
Copied S047
Copied S010
Copied S009
Copied S084
Copied S114
Copied S086
Copied S061
Copied S054
Copied S018
Copied S101
Copied S082
Copied S041
Copied S0